In [1]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.styles import PatternFill, Font, Alignment, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows

In [2]:
%%capture
%run "8. Implementación.ipynb"

In [3]:
def Exportar_Resultados_A_Excel_Con_Formato(Evaluaciones_Modelos, Alpha_Significancia=0.05, 
                                           Nombre_Archivo='Resultados_Modelos_Robustos.xlsx'):

    """
    Exporta resultados de modelos robustos a Excel con formato condicional,
    mostrando coeficientes beta y p-valores en una tabla matricial con
    colores que indican significancia estadística.
    
    Extrae automáticamente todas las variables del diccionario de evaluaciones
    y crea una tabla donde las filas son variables independientes, las columnas
    son variables dependientes, y cada celda muestra β y p-valor con color
    verde para significativas y gris para no significativas.
    
    Parámetros:
    -----------
    Evaluaciones_Modelos : dict
        Diccionario con resultados de evaluaciones de modelos robustos.
        Estructura: {Variable_Dep: {'Interpretaciones_Coeficientes': {...}}}
    
    Alpha_Significancia : float, opcional (default=0.05)
        Nivel de significancia para determinar color de celdas.
    
    Nombre_Archivo : str, opcional (default='Resultados_Modelos_Robustos.xlsx')
        Nombre del archivo Excel a generar.
    
    Retorna:
    --------
    pandas.DataFrame
        DataFrame con los resultados que se exportó a Excel.
    
    """
    
    print(f"📊 Extrayendo datos del diccionario de evaluaciones...")
    
    # PASO 1: Extraer automáticamente todas las variables.
    Variables_Dependientes = []
    Variables_Independientes = set()
    
    # Recorrer el diccionario como lo hiciste tú.
    for Var_Dep, Dict_Var_Dep in Evaluaciones_Modelos.items():
        if Dict_Var_Dep is not None and 'Interpretaciones_Coeficientes' in Dict_Var_Dep:
            Variables_Dependientes.append(Var_Dep)
            
            for Var_Indep, Dict_Var_Indep in Dict_Var_Dep['Interpretaciones_Coeficientes'].items():
                Variables_Independientes.add(Var_Indep)
    
    Variables_Independientes = sorted(list(Variables_Independientes))
    
    # print(f"   Variables dependientes encontradas: {len(Variables_Dependientes)}")
    # print(f"   Variables independientes encontradas: {len(Variables_Independientes)}")
    
    # PASO 2: Crear DataFrame vacío.
    Datos_Tabla = pd.DataFrame(
        index=Variables_Independientes,
        columns=Variables_Dependientes
    )
    
    # PASO 3: Rellenar datos extraídos del diccionario.
    for Var_Dep, Dict_Var_Dep in Evaluaciones_Modelos.items():
        if Dict_Var_Dep is not None and 'Interpretaciones_Coeficientes' in Dict_Var_Dep:
            
            for Var_Indep, Dict_Var_Indep in Dict_Var_Dep['Interpretaciones_Coeficientes'].items():
                Beta = Dict_Var_Indep['Beta']
                P_Valor = Dict_Var_Indep['P_Valor']
                
                # Formatear texto de la celda.
                if P_Valor < 0.001:
                    Texto_Celda = f"β = {Beta:.3f}\np < 0.001"
                else:
                    Texto_Celda = f"β = {Beta:.3f}\np = {P_Valor:.3f}"
                
                Datos_Tabla.loc[Var_Indep, Var_Dep] = Texto_Celda
    
    # Rellenar celdas vacías con guión.
    Datos_Tabla = Datos_Tabla.fillna("-")
    
    print(f"📈 Generando archivo Excel con formato...")
    
    # PASO 4: Crear workbook de Excel con formato.
    Libro = Workbook()
    Worksheet = Libro.active
    Worksheet.title = "Resultados Modelos Robustos"
    
    # PASO 5: Configurar estilos.
    # Colores.
    Color_Significativo = PatternFill(start_color="28A745", end_color="28A745", fill_type="solid")  # Verde
    Color_No_Significativo = PatternFill(start_color="6C757D", end_color="6C757D", fill_type="solid")  # Gris
    Color_Encabezado = PatternFill(start_color="F8F9FA", end_color="F8F9FA", fill_type="solid")  # Gris claro
    
    # Fuentes.
    Fuente_Encabezado = Font(bold=True, size=10)
    Fuente_Celda_Significativa = Font(color="FFFFFF", size=9, bold=True)  # Blanco sobre verde
    Fuente_Celda_No_Significativa = Font(color="FFFFFF", size=9)  # Blanco sobre gris
    
    # Alineación.
    Alineacion_Centro = Alignment(horizontal="center", vertical="center", wrap_text=True)
    Alineacion_Izquierda = Alignment(horizontal="left", vertical="center")
    
    # Bordes.
    Borde_Fino = Border(
        left=Side(style="thin"), right=Side(style="thin"),
        top=Side(style="thin"), bottom=Side(style="thin")
    )
    
    # PASO 6: Escribir encabezados de columnas (variables dependientes).
    for j, Var_Dep in enumerate(Variables_Dependientes, start=2):
        Celda = Worksheet.cell(row=1, column=j)
        # Acortar nombres largos.
        Nombre_Corto = Var_Dep.replace('CO_Item_', '').replace('_Izq', ' (I)').replace('_Der', ' (D)')
        Celda.value = Nombre_Corto
        Celda.fill = Color_Encabezado
        Celda.font = Fuente_Encabezado
        Celda.alignment = Alineacion_Centro
        Celda.border = Borde_Fino
        Worksheet.column_dimensions[Celda.column_letter].width = 15
    
    # PASO 7: Escribir encabezados de filas (variables independientes) y datos.
    for i, Var_Indep in enumerate(Variables_Independientes, start=2):
        
        # Encabezado de fila.
        Celda_Encabezado = Worksheet.cell(row=i, column=1)
        # Acortar nombres largos.
        Nombre_Corto = Var_Indep.replace('Categoria_PASO_2023_', '').replace('Medios_Prensa_', '').replace('Autopercepcion_', '')
        if len(Nombre_Corto) > 25:
            Nombre_Corto = Nombre_Corto[:22] + "..."
        Celda_Encabezado.value = Nombre_Corto
        Celda_Encabezado.fill = Color_Encabezado
        Celda_Encabezado.font = Fuente_Encabezado
        Celda_Encabezado.alignment = Alineacion_Izquierda
        Celda_Encabezado.border = Borde_Fino
        
        # Datos de cada variable dependiente.
        for j, Var_Dep in enumerate(Variables_Dependientes, start=2):
            Celda_Datos = Worksheet.cell(row=i, column=j)
            Valor_Celda = Datos_Tabla.loc[Var_Indep, Var_Dep]
            
            if Valor_Celda == "-":
                # Celda sin datos.
                Celda_Datos.value = "-"
                Celda_Datos.fill = Color_No_Significativo
                Celda_Datos.font = Fuente_Celda_No_Significativa
            else:
                # Celda con datos - determinar si es significativa.
                # Extraer p-valor del texto.
                if "p < 0.001" in Valor_Celda:
                    Es_Significativo = True  # p < 0.001 siempre es significativo
                else:
                    # Extraer p-valor del texto.
                    try:
                        Lineas = Valor_Celda.split('\n')
                        P_Linea = [linea for linea in Lineas if 'p =' in linea][0]
                        P_Valor_Extraido = float(P_Linea.split('p = ')[1])
                        Es_Significativo = P_Valor_Extraido < Alpha_Significancia
                    except:
                        Es_Significativo = False
                
                Celda_Datos.value = Valor_Celda
                
                if Es_Significativo:
                    Celda_Datos.fill = Color_Significativo
                    Celda_Datos.font = Fuente_Celda_Significativa
                else:
                    Celda_Datos.fill = Color_No_Significativo
                    Celda_Datos.font = Fuente_Celda_No_Significativa
            
            Celda_Datos.alignment = Alineacion_Centro
            Celda_Datos.border = Borde_Fino
    
    # PASO 8: Ajustar dimensiones.
    Worksheet.column_dimensions['A'].width = 30  # Columna de variables independientes más ancha
    Worksheet.row_dimensions[1].height = 25  # Fila de encabezados más alta
    
    # Ajustar altura de filas con datos.
    for i in range(2, len(Variables_Independientes) + 2):
        Worksheet.row_dimensions[i].height = 35
    
    # PASO 9: Agregar leyenda.
    Fila_Leyenda = len(Variables_Independientes) + 4
    
    # Título de leyenda.
    Celda_Titulo_Leyenda = Worksheet.cell(row=Fila_Leyenda, column=1)
    Celda_Titulo_Leyenda.value = "LEYENDA:"
    Celda_Titulo_Leyenda.font = Font(bold=True, size=12)
    
    # Leyenda significativo.
    Celda_Leyenda_Sig = Worksheet.cell(row=Fila_Leyenda + 1, column=1)
    Celda_Leyenda_Sig.value = "Variables significativas"
    Celda_Leyenda_Sig.fill = Color_Significativo
    Celda_Leyenda_Sig.font = Fuente_Celda_Significativa
    Celda_Leyenda_Sig.border = Borde_Fino
    
    Celda_Texto_Sig = Worksheet.cell(row=Fila_Leyenda + 1, column=2)
    Celda_Texto_Sig.value = f"p < {Alpha_Significancia}"
    
    # Leyenda no significativo.
    Celda_Leyenda_No_Sig = Worksheet.cell(row=Fila_Leyenda + 2, column=1)
    Celda_Leyenda_No_Sig.value = "Variables no significativas"
    Celda_Leyenda_No_Sig.fill = Color_No_Significativo
    Celda_Leyenda_No_Sig.font = Fuente_Celda_No_Significativa
    Celda_Leyenda_No_Sig.border = Borde_Fino
    
    Celda_Texto_No_Sig = Worksheet.cell(row=Fila_Leyenda + 2, column=2)
    Celda_Texto_No_Sig.value = f"p >= {Alpha_Significancia} o no incluidas"
    
    # PASO 10: Agregar información adicional.
    Fila_Info = Fila_Leyenda + 5
    Worksheet.cell(row=Fila_Info, column=1).value = f"Total variables dependientes: {len(Variables_Dependientes)}"
    Worksheet.cell(row=Fila_Info + 1, column=1).value = f"Total variables independientes: {len(Variables_Independientes)}"
    
    # Contar celdas significativas.
    Total_Celdas = len(Variables_Independientes) * len(Variables_Dependientes)
    Celdas_Con_Datos = (Datos_Tabla != "-").sum().sum()
    
    # Contar significativas.
    Celdas_Significativas = 0
    for Var_Dep, Dict_Var_Dep in Evaluaciones_Modelos.items():
        if Dict_Var_Dep is not None and 'Interpretaciones_Coeficientes' in Dict_Var_Dep:
            for Var_Indep, Dict_Var_Indep in Dict_Var_Dep['Interpretaciones_Coeficientes'].items():
                if Dict_Var_Indep['P_Valor'] < Alpha_Significancia:
                    Celdas_Significativas += 1
    
    Worksheet.cell(row=Fila_Info + 2, column=1).value = f"Celdas con datos: {Celdas_Con_Datos}/{Total_Celdas} ({Celdas_Con_Datos/Total_Celdas*100:.1f}%)"
    Worksheet.cell(row=Fila_Info + 3, column=1).value = f"Celdas significativas: {Celdas_Significativas}/{Total_Celdas} ({Celdas_Significativas/Total_Celdas*100:.1f}%)"
    
    # PASO 11: Guardar archivo.
    try:
        Libro.save(Nombre_Archivo)
        # print(f"✅ Archivo Excel guardado exitosamente: {Nombre_Archivo}")
        # print(f"📊 Estadísticas:")
        # print(f"   Total de celdas: {Total_Celdas}")
        # print(f"   Celdas con datos: {Celdas_Con_Datos} ({Celdas_Con_Datos/Total_Celdas*100:.1f}%)")
        # print(f"   Celdas significativas: {Celdas_Significativas} ({Celdas_Significativas/Total_Celdas*100:.1f}%)")
        
    except Exception as e:
        print(f"❌ Error al guardar archivo Excel: {str(e)}")
        raise
    
    return Datos_Tabla

In [4]:
# Directo con tu diccionario
Tabla_Resultados = Exportar_Resultados_A_Excel_Con_Formato(
    Evaluaciones_Modelos=Evaluaciones_Modelos,  
    Alpha_Significancia=0.05,
    Nombre_Archivo='Resultados_Ballotage.xlsx'
)

📊 Extrayendo datos del diccionario de evaluaciones...
📈 Generando archivo Excel con formato...


In [5]:
def Imprimir_Variables_Significativas(Evaluaciones_Modelos, Resultados_Modelos=None, 
                                     Alpha_Significancia=0.05, Ordenar_Por_P_Valor=True, 
                                     Mostrar_Metricas=True):

    """
    Imprime de forma clara las variables significativas para cada modelo,
    mostrando la variable dependiente, métricas del modelo, seguida de un listado 
    con guiones de las variables independientes significativas con sus coeficientes
    y p-valores en el mismo renglón.
    
    Parámetros:
    -----------
    Evaluaciones_Modelos : dict
        Diccionario con resultados de evaluaciones de modelos robustos.
    
    Resultados_Modelos : dict, opcional (default=None)
        Diccionario con resultados originales del modelo robusto que contiene
        métricas como AIC, BIC, R², etc. Si no se proporciona, usa métricas
        disponibles en Evaluaciones_Modelos.
    
    Alpha_Significancia : float, opcional (default=0.05)
        Nivel de significancia para filtrar variables.
    
    Ordenar_Por_P_Valor : bool, opcional (default=True)
        Si True, ordena las variables significativas por p-valor ascendente.
        Si False, las muestra en el orden que aparecen en el diccionario.
    
    Mostrar_Metricas : bool, opcional (default=True)
        Si True, muestra métricas del modelo (R², AIC, BIC, etc.).
        Si False, solo muestra variables significativas.
    
    """
    
    Total_Variables_Dependientes = 0
    Total_Variables_Significativas = 0
    
    for Var_Dep, Dict_Var_Dep in Evaluaciones_Modelos.items():
        if Dict_Var_Dep is not None and 'Interpretaciones_Coeficientes' in Dict_Var_Dep:
            
            Total_Variables_Dependientes += 1
            
            # Imprimir variable dependiente.
            print(f"{Var_Dep}:")
            
            # Mostrar métricas del modelo si están disponibles y se solicita.
            if Mostrar_Metricas:
                Metricas = []
                
                # Buscar métricas en Resultados_Modelos primero (más completo).
                if Resultados_Modelos and Var_Dep in Resultados_Modelos and Resultados_Modelos[Var_Dep] is not None:
                    Resultado_Modelo = Resultados_Modelos[Var_Dep]
                    
                    if 'R_Cuadrado_Ajustado' in Resultado_Modelo:
                        Metricas.append(f"R² = {Resultado_Modelo['R_Cuadrado_Ajustado']:.3f}")
                    if 'AIC' in Resultado_Modelo:
                        Metricas.append(f"AIC = {Resultado_Modelo['AIC']:.1f}")
                    if 'BIC' in Resultado_Modelo:
                        Metricas.append(f"BIC = {Resultado_Modelo['BIC']:.1f}")
                    if 'P_Valor_Modelo' in Resultado_Modelo:
                        P_Modelo = Resultado_Modelo['P_Valor_Modelo']
                        P_Modelo_Str = "p < 0.001" if P_Modelo < 0.001 else f"p = {P_Modelo:.4f}"
                        Metricas.append(f"F-test: {P_Modelo_Str}")
                    if 'N_Observaciones' in Resultado_Modelo:
                        Metricas.append(f"N = {int(Resultado_Modelo['N_Observaciones'])}")
                
                # Si no hay métricas en Resultados_Modelos, buscar en Evaluaciones_Modelos.
                if not Metricas:
                    if 'Calidad_Ajuste' in Dict_Var_Dep:
                        Metricas.append(f"Calidad: {Dict_Var_Dep['Calidad_Ajuste']}")
                    if 'Validez_General' in Dict_Var_Dep:
                        Metricas.append(f"Validez: {Dict_Var_Dep['Validez_General']}")
                    if 'Puntuacion_Confianza' in Dict_Var_Dep:
                        Metricas.append(f"Confianza: {Dict_Var_Dep['Puntuacion_Confianza']:.0f}%")
                
                if Metricas:
                    print(f"  📊 {' | '.join(Metricas)}")
            
            # Recopilar variables significativas para esta variable dependiente.
            Variables_Significativas = []
            
            for Var_Indep, Dict_Var_Indep in Dict_Var_Dep['Interpretaciones_Coeficientes'].items():
                Beta = Dict_Var_Indep['Beta']
                P_Valor = Dict_Var_Indep['P_Valor']
                
                if P_Valor < Alpha_Significancia:
                    Variables_Significativas.append({
                        'Variable': Var_Indep,
                        'Beta': Beta,
                        'P_Valor': P_Valor
                    })
            
            # Ordenar por p-valor si se solicita.
            if Ordenar_Por_P_Valor and Variables_Significativas:
                Variables_Significativas.sort(key=lambda x: x['P_Valor'])
            
            # Imprimir variables significativas.
            if Variables_Significativas:
                Total_Variables_Significativas += len(Variables_Significativas)
                
                for var_sig in Variables_Significativas:
                    Beta = var_sig['Beta']
                    P_Val = var_sig['P_Valor']
                    Var_Nombre = var_sig['Variable']
                    
                    # Formatear p-valor.
                    if P_Val < 0.001:
                        P_Formato = "p < 0.001"
                    else:
                        P_Formato = f"p = {P_Val:.3f}"
                    
                    print(f"  - {Var_Nombre}: β = {Beta:.3f}, {P_Formato}")
                    
            else:
                print("  (Sin variables significativas)")
            
            print()  # Línea en blanco entre variables dependientes
    
    # Resumen final.
    print(f"RESUMEN:")
    print(f"Variables dependientes analizadas: {Total_Variables_Dependientes}")
    print(f"Total de relaciones significativas: {Total_Variables_Significativas}")

In [6]:
Imprimir_Variables_Significativas(Evaluaciones_Modelos,
                                  Resultados_Modelos)

CO_Item_6_Izq:
  📊 R² = 0.064 | AIC = 1441.1 | BIC = 1495.1 | F-test: p = 0.0035 | N = 469
  - Indice_Progresismo: β = -0.216, p = 0.019
  - Genero_Masculino: β = 0.225, p = 0.036
  - Medios_Prensa_Pagina_12: β = 0.282, p = 0.038

CO_Item_25_Izq:
  📊 R² = 0.091 | AIC = 1266.8 | BIC = 1319.6 | F-test: p = 0.0022 | N = 429
  - Indice_Progresismo: β = -0.333, p = 0.009
  - Medios_Prensa_El_Cronista: β = -0.435, p = 0.011
  - Medios_Prensa_Prensa_Obrera: β = 0.411, p = 0.013
  - Region_Patagonia: β = -0.598, p = 0.015
  - Medios_Prensa_Perfil: β = 0.336, p = 0.032
  - Categoria_PASO_2023_Moderate_Right_B: β = 0.513, p = 0.035
  - Autopercepcion_Per_Antiper: β = -0.051, p = 0.050

CO_Item_25_Der:
  📊 R² = 0.089 | AIC = 1337.8 | BIC = 1374.4 | F-test: p < 0.001 | N = 429
  - Indice_Progresismo: β = -0.411, p < 0.001
  - Autopercepcion_Per_Antiper: β = -0.072, p = 0.006
  - Medios_Prensa_Diario_Universal: β = 0.636, p = 0.033

CO_Item_11_Der:
  📊 R² = 0.106 | AIC = 1357.0 | BIC = 1438.3 | F-t

In [7]:
def Imprimir_Top_Predictores(Evaluaciones_Modelos, Alpha_Significancia=0.05, Top_N=10):

    """
    Muestra los predictores más frecuentes y significativos a través de todos los modelos.
    
    """
    
    # Contar frecuencia de cada predictor significativo.
    Contador_Predictores = {}
    
    for Var_Dep, Dict_Var_Dep in Evaluaciones_Modelos.items():
        if Dict_Var_Dep is not None and 'Interpretaciones_Coeficientes' in Dict_Var_Dep:
            
            for Var_Indep, Dict_Var_Indep in Dict_Var_Dep['Interpretaciones_Coeficientes'].items():
                P_Valor = Dict_Var_Indep['P_Valor']
                Beta = Dict_Var_Indep['Beta']
                
                if P_Valor < Alpha_Significancia:
                    if Var_Indep not in Contador_Predictores:
                        Contador_Predictores[Var_Indep] = {
                            'Frecuencia': 0,
                            'Betas_Promedio': [],
                            'P_Valores_Promedio': []
                        }
                    
                    Contador_Predictores[Var_Indep]['Frecuencia'] += 1
                    Contador_Predictores[Var_Indep]['Betas_Promedio'].append(Beta)
                    Contador_Predictores[Var_Indep]['P_Valores_Promedio'].append(P_Valor)
    
    # Ordenar por frecuencia.
    Predictores_Ordenados = sorted(
        Contador_Predictores.items(), 
        key=lambda x: x[1]['Frecuencia'], 
        reverse=True
    )
    
    print(f"TOP {Top_N} PREDICTORES MÁS FRECUENTES:")
    print("="*50)
    
    for i, (Predictor, Datos) in enumerate(Predictores_Ordenados[:Top_N], 1):
        Frecuencia = Datos['Frecuencia']
        Beta_Promedio = sum(Datos['Betas_Promedio']) / len(Datos['Betas_Promedio'])
        P_Promedio = sum(Datos['P_Valores_Promedio']) / len(Datos['P_Valores_Promedio'])
        
        print(f"{i:2d}. {Predictor}")
        print(f"    Aparece en {Frecuencia} modelos | β promedio = {Beta_Promedio:.3f} | p promedio = {P_Promedio:.4f}")
        print()

In [8]:
Imprimir_Top_Predictores(Evaluaciones_Modelos, Alpha_Significancia=0.05)

TOP 10 PREDICTORES MÁS FRECUENTES:
 1. Indice_Progresismo
    Aparece en 5 modelos | β promedio = -0.333 | p promedio = 0.0120

 2. Indice_Conservadurismo
    Aparece en 4 modelos | β promedio = -0.430 | p promedio = 0.0013

 3. Autopercepcion_Per_Antiper
    Aparece en 3 modelos | β promedio = -0.059 | p promedio = 0.0238

 4. Categoria_PASO_2023_Moderate_Right_A
    Aparece en 3 modelos | β promedio = -0.597 | p promedio = 0.0277

 5. Medios_Prensa_Pagina_12
    Aparece en 2 modelos | β promedio = -0.011 | p promedio = 0.0304

 6. Progressive_Cluster
    Aparece en 2 modelos | β promedio = -0.002 | p promedio = 0.0326

 7. Influencia_Redes
    Aparece en 2 modelos | β promedio = -0.004 | p promedio = 0.0275

 8. Indice_Conservadurismo_Tiempo
    Aparece en 2 modelos | β promedio = 0.020 | p promedio = 0.0244

 9. Genero_Masculino
    Aparece en 1 modelos | β promedio = 0.225 | p promedio = 0.0355

10. Region_Patagonia
    Aparece en 1 modelos | β promedio = -0.598 | p promedio = 0.01